# SQL ALCHEMY

## Import des librairies nécessaires pour la connexion SQL et la manipulation des données

In [15]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime

## Chemin vers le fichier de base de données SQLite

In [16]:
db_file = "sakila.db"
engine = create_engine(f"sqlite:///{db_file}")

## Exécute une requête SQL sur une base SQLite

In [17]:
def executer_query(sql_query, params=None):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query), params or {})
            if result.returns_rows:
                df = pd.DataFrame(result.fetchall(), columns=result.keys())
                return df
            else:
                connection.commit()  # commit nécessaire pour SQLite
                return None
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return None

## Affichez les **5 titres de films** (`title` de la table `film`) classés par **ordre alphabétique croissant**, en utilisant un **alias** pour la colonne.

In [18]:
sql_titles = """
SELECT title AS "Titre du film"
FROM film
ORDER BY title ASC
LIMIT 5;
"""
df_titles = executer_query(sql_titles)
print(df_titles.to_markdown(index=False))

| Titre du film    |
|:-----------------|
| ACADEMY DINOSAUR |
| ACE GOLDFINGER   |
| ADAPTATION HOLES |
| AFFAIR PREJUDICE |
| AFRICAN EGG      |


## Calculez la **moyenne**, le **minimum**, le **maximum**, le **nombre total** et la **somme** du champ `length` (durée en minutes) de tous les films dans la table `film`.

In [19]:
sql_stats = """
SELECT 
    AVG(length) AS "Durée Moyenne",
    MIN(length) AS "Durée Min",
    MAX(length) AS "Durée Max",
    COUNT(length) AS "Nombre de Films",
    SUM(length) AS "Somme des Durées"
FROM film;
"""
df_stats = executer_query(sql_stats)
print(df_stats.to_markdown(index=False))

|   Durée Moyenne |   Durée Min |   Durée Max |   Nombre de Films |   Somme des Durées |
|----------------:|------------:|------------:|------------------:|-------------------:|
|         115.272 |          46 |         185 |              1000 |             115272 |


## **Insérez** un nouvel acteur avec le prénom `"JEAN-CLAUDE"`, le nom `"VANDAMME"`, le nouvel ID calculé, et la date/heure actuelle pour le champ **`last_update`**.
      * **Trouvez** l'**`actor_id` maximum** existant dans la table `actor`, puis **calculez le nouvel ID** en ajoutant 1.
      * **Format de date requis pour le `last_update` :** Utilisez le module `datetime` pour générer la chaîne `YYYY-MM-DD HH:MM:SS`.

In [20]:
first_name = "JEAN-CLAUDE"
last_name = "VANDAMME"

# Vérifier si l'acteur existe déjà
sql_check = """
SELECT COUNT(*) AS count
FROM actor
WHERE first_name = :first_name AND last_name = :last_name;
"""
df_check = executer_query(sql_check, {"first_name": first_name, "last_name": last_name})

if df_check.iat[0, 0] > 0:
    print(f"L'acteur '{first_name} {last_name}' existe déjà. Aucune insertion effectuée.")
else:
    # Trouver l'actor_id maximum existant et calculer le nouvel ID
    df_max_id = executer_query("SELECT MAX(actor_id) AS max_id FROM actor;")
    raw_id = df_max_id.iat[0, 0] or 0
    nouvel_id = int(raw_id) + 1

    # Préparer les données à insérer
    nouvel_acteur = {
        "actor_id": nouvel_id,
        "first_name": first_name,
        "last_name": last_name,
        "last_update": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    # Insérer le nouvel acteur
    sql_insert = """
    INSERT INTO actor (actor_id, first_name, last_name, last_update)
    VALUES (:actor_id, :first_name, :last_name, :last_update);
    """
    executer_query(sql_insert, nouvel_acteur)
    print(f"L'acteur '{first_name} {last_name}' a été inséré avec l'ID {nouvel_id}.")

L'acteur 'JEAN-CLAUDE VANDAMME' a été inséré avec l'ID 205.


**Modification (`UPDATE`) par ID :** **Modifiez** l'acteur que vous venez d'insérer (en le ciblant par son **ID unique**) pour changer son prénom de `"JEAN-CLAUDE"` à `"JC"`.

In [21]:
sql_update = """
UPDATE actor
SET first_name = :new_first_name, last_update = :last_update
WHERE actor_id = :actor_id;
"""
executer_query(sql_update, {
    "new_first_name": "JC",
    "last_update": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "actor_id": nouvel_id
})

# Vérifier la modification
sql_verif = "SELECT actor_id, first_name, last_name FROM actor WHERE actor_id = :actor_id;"
df_verif = executer_query(sql_verif, {"actor_id": nouvel_id})
print(df_verif.to_markdown(index=False))

|   actor_id | first_name   | last_name   |
|-----------:|:-------------|:------------|
|        205 | JC           | VANDAMME    |
